In [ ]:
%load_ext autoreload
%load_ext tensorboard
%matplotlib inline

# Measuring Effects of Interventions in CBM for 3dshapes

## Setup

In [ ]:
import matplotlib
import concepts_xai
import numpy as np
import os
import random
import tensorflow as tf
import yaml
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import rc
from matplotlib import cm
import seaborn as sns
from importlib import reload
from pathlib import Path
import sklearn
import scipy
from sklearn.model_selection import train_test_split
import utils
import model_utils

In [ ]:
################################################################################
## Set seeds up for reproducibility
################################################################################

os.environ['PYTHONHASHSEED'] = str(87)
tf.random.set_seed(87)
np.random.seed(87)
random.seed(87)
NUM_TRIALS = 5

In [ ]:
################################################################################
## Global Variables Defining Experiment Flow
################################################################################

_LATEX_SYMBOL = ""  # Change to "$" if working out of server
USE_DSPRITES = True
if USE_DSPRITES:
    RESULTS_DIR = "intervention_results/dsprites"
    DATASETS_DIR = os.path.join("results/dsprites", "datasets/")
else:
    RESULTS_DIR = "intervention_results/shapes3d"
    DATASETS_DIR = os.path.join("results/shapes3d", "datasets/")
    
Path(DATASETS_DIR).mkdir(parents=True, exist_ok=True)
rc('text', usetex=(_LATEX_SYMBOL == "$"))
plt.style.use('seaborn-whitegrid')


## Dependency Tasks Dataset Construction

In [ ]:
if not USE_DSPRITES:
    import tensorflow_datasets as tfds

    shapes3d_train_ds = tfds.load('shapes3d', split='train', shuffle_files=True)
else:
    import concepts_xai.datasets.dSprites as dsprites
    import concepts_xai.datasets.latentFactorData as latentFactorData

In [ ]:
def relabel_categorical(x):
    if len(x.shape) == 1:
        _, unique_inds = np.unique(
            x,
            return_inverse=True,
        )
        return unique_inds
    
    result = x[:, :]
    for i in range(x.shape[-1]):
        _, unique_inds = np.unique(
            x[:, i],
            return_inverse=True,
        )
        result[:, i] = unique_inds
    return result

def cardinality_encoding(card_group_1, card_group_2):
    result_to_encoding = {}
    for i in card_group_1:
        for j in card_group_2:
            result_to_encoding[(i, j)] = len(result_to_encoding)
    return result_to_encoding

def extract_data(
    filter_fn,
    sample_map_fn=lambda x: x,
    concept_map_fn=lambda x: x,
    step=1,
    label_fn=lambda ex: ex['label_shape'] * 8 + ex['label_scale'],
    dataset_path=None,
    force_rerun=False,
):
    if (not force_rerun) and dataset_path and os.path.exists(dataset_path):
        # Them time to load up this dataset!
        ds = np.load(dataset_path)
        return ds["X"], ds["y"], ds["c"]
    num_entries = len(shapes3d_train_ds)
    x_train = []
    y_train = []
    c_train = []
    for i, ex in enumerate(tfds.as_numpy(
        shapes3d_train_ds.shuffle(buffer_size=15000)
    )):
        if i % step != 0:
            continue
        concepts = [
            ex['label_floor_hue'],
            ex['label_wall_hue'],
            ex['label_object_hue'],
            ex['label_scale'],
            ex['label_shape'],
            ex['label_orientation'],
        ]
        if not filter_fn(concepts):
            continue
        print(i, end="\r")

        x_train.append(sample_map_fn(ex['image']))
        y_train.append(label_fn(ex))
        c_train.append(concept_map_fn(concepts))
    x_train = np.stack(x_train, axis=0) / 255.0
    y_train = relabel_categorical(np.stack(y_train, axis=0))
    c_train = relabel_categorical(np.stack(c_train, axis=0))
    if dataset_path:
        # Then serialize it to speed up things next time
        np.savez(
            dataset_path,
            X=x_train,
            y=y_train,
            c=c_train,
        )
    return x_train, y_train, c_train


def generate_dsprites_dataset(
    label_fn,
    filter_fn=None,
    dataset_path=None,
    concept_map_fn=lambda x: x,
    sample_map_fn=lambda x: x,
    dsprites_path="dsprites/dsprites_ndarray_co1sh3sc6or40x32y32_64x64.npz",
    force_reload=False,
):
    if (not force_reload) and dataset_path and os.path.exists(dataset_path):
        # Them time to load up this dataset!
        ds = np.load(dataset_path)
        return (
            (ds["x_train"], ds["y_train"], ds["c_train"]),
            (ds["x_test"], ds["y_test"], ds["c_test"])
        )
    
    def _task_fn(x_data, c_data):
        return latentFactorData.get_task_data(
            x_data=x_data,
            c_data=c_data,
            label_fn=label_fn,
            filter_fn=filter_fn,
        )

    loaded_dataset = dsprites.dSprites(
        dataset_path=dsprites_path,
        train_size=0.8,
        random_state=42,
        task=_task_fn,
    )
    _, _, _ = loaded_dataset.load_data()

    x_train = sample_map_fn(loaded_dataset.x_train)
    y_train = loaded_dataset.y_train
    c_train = concept_map_fn(loaded_dataset.c_train)
    
    x_test = sample_map_fn(loaded_dataset.x_test)
    y_test = loaded_dataset.y_test
    c_test = concept_map_fn(loaded_dataset.c_test)
    
    if dataset_path:
        # Then serialize it to speed up things next time
        np.savez(
            dataset_path,
            x_train=x_train,
            y_train=y_train,
            c_train=c_train,
            x_test=x_test,
            y_test=y_test,
            c_test=c_test,
        )
    return (x_train, y_train, c_train), (x_test, y_test, c_test),

## Dataset Construction

In [ ]:
if not USE_DSPRITES:
    ############################################################################
    ## Construct a binary concept task in the shapes3D dataset 
    ############################################################################
    def count_class_balance(y):
        one_hot = tf.keras.utils.to_categorical(y)
        return np.sum(one_hot, axis=0) / one_hot.shape[0]

    def multiclass_task_bin_concepts_map_fn(concepts):
        return [
            int(concepts[0] < 5),
            int(concepts[1] < 5),
            int(concepts[2] < 5),
            int(concepts[3] < 4),
            int(concepts[4] < 2),
            int(concepts[5] < 7),
        ]


    def multiclass_task_bin_concepts_label_fn(concept_dict):
        concept_vector = multiclass_task_bin_concepts_map_fn([
            concept_dict['label_floor_hue'],
            concept_dict['label_wall_hue'],
            concept_dict['label_object_hue'],
            concept_dict['label_scale'],
            concept_dict['label_shape'],
            concept_dict['label_orientation'],
        ])
        binary_label_encoding = [
            concept_vector[0] or concept_vector[1],
            concept_vector[2] or concept_vector[3],
            concept_vector[4] or concept_vector[5],
        ]
        return int(
            "".join(list(map(str, binary_label_encoding))),
            2
        )

    def filter_fn_dep_0(concept):
        ranges = [
            list(range(0, 10, 2)),
            list(range(0, 10, 2)),
            list(range(0, 10, 2)),
            list(range(0, 8)),
            list(range(4)),
            list(range(0, 15, 4)),
        ]

        concept_vector = multiclass_task_bin_concepts_map_fn(concept)
        # First filter as in small dataset to constraint the size of the data a bit
        return all([
            (concept[i] in ranges[i]) for i in range(len(ranges))
        ])



    floor_hue_wall_hue_sets_lower = [
        list(np.random.permutation(7))[:5]
        for _ in range(10)
    ]

    floor_hue_wall_hue_sets_upper = [
        list(3 + np.random.permutation(7))[:5]
        for _ in range(10)
    ]

    def filter_fn_dep_1(concept):
        ranges = [
            list(range(0, 10)),
            list(range(0, 10, 2)),
            list(range(0, 10, 2)),
            list(range(0, 8)),
            list(range(4)),
            list(range(0, 15, 4)),
        ]

        concept_vector = multiclass_task_bin_concepts_map_fn(concept)

        # First filter as in small dataset to constraint the size of the data a bit
        if not all([
            (concept[i] in ranges[i]) for i in range(len(ranges))
        ]):
            return False
        if concept[0] < 5:
            return concept[1] in floor_hue_wall_hue_sets_lower[concept[0]]
        else:
            return (concept[1] in floor_hue_wall_hue_sets_upper[concept[0]])


        wall_hue_object_hue_sets_lower = [
        list(np.random.permutation(7))[:5]
        for _ in range(10)
    ]
    wall_hue_object_hue_sets_upper = [
        list(3 + np.random.permutation(7))[:5]
        for _ in range(10)
    ]

    def filter_fn_dep_2(concept):
        ranges = [
            list(range(0, 10)),
            list(range(0, 10)),
            list(range(0, 10, 2)),
            list(range(0, 8)),
            list(range(4)),
            list(range(0, 15, 4)),
        ]

        concept_vector = multiclass_task_bin_concepts_map_fn(concept)
        # First filter as in small dataset to constraint the size of the data a bit
        if not all([
            (concept[i] in ranges[i]) for i in range(len(ranges))
        ]):
            return False

        if (concept[0] < 5):
            if concept[1] not in floor_hue_wall_hue_sets_lower[concept[0]]:
                return False
        else:
            if (concept[1] not in floor_hue_wall_hue_sets_upper[concept[0]]):
                return False

        if (concept[1] < 5):
            if concept[2] not in wall_hue_object_hue_sets_lower[concept[1]]:
                return False
        else:
            if (concept[2] not in wall_hue_object_hue_sets_upper[concept[1]]):
                return False
        return True


        object_hue_scale_sets_lower = [
        list(np.random.permutation(6))[:4]
        for _ in range(10)
    ]

    object_hue_scale_sets_upper = [
        list(2 + np.random.permutation(6))[:4]
        for _ in range(10)
    ]

    def filter_fn_dep_3(concept):
        ranges = [
            list(range(0, 10)),
            list(range(0, 10)),
            list(range(0, 10)),
            list(range(0, 8)),
            list(range(4)),
            list(range(0, 15, 4)),
        ]

        concept_vector = multiclass_task_bin_concepts_map_fn(concept)

        # First filter as in small dataset to constraint the size of the data a bit
        if not all([
            (concept[i] in ranges[i]) for i in range(len(ranges))
        ]):
            return False

        if (concept[0] < 5):
            if concept[1] not in floor_hue_wall_hue_sets_lower[concept[0]]:
                return False
        else:
            if (concept[1] not in floor_hue_wall_hue_sets_upper[concept[0]]):
                return False

        if (concept[1] < 5):
            if concept[2] not in wall_hue_object_hue_sets_lower[concept[1]]:
                return False
        else:
            if (concept[2] not in wall_hue_object_hue_sets_upper[concept[1]]):
                return False

        if (concept[2] < 5):
            if concept[3] not in object_hue_scale_sets_lower[concept[2]]:
                return False
        else:
            if (concept[3] not in object_hue_scale_sets_upper[concept[2]]):
                return False

        return True


    scale_shape_sets_lower = [
        list(np.random.permutation(3))[:2]
        for _ in range(8)
    ]

    scale_shape_sets_upper = [
        list(1 + np.random.permutation(3))[:2]
        for _ in range(8)
    ]

    def filter_fn_dep_4(concept):
        ranges = [
            list(range(0, 10)),
            list(range(0, 10)),
            list(range(0, 10)),
            list(range(0, 8)),
            list(range(4)),
            list(range(0, 15, 2)),
        ]

        concept_vector = multiclass_task_bin_concepts_map_fn(concept)

        # First filter as in small dataset to constraint the size of the data a bit
        if not all([
            (concept[i] in ranges[i]) for i in range(len(ranges))
        ]):
            return False

        if (concept[0] < 5):
            if concept[1] not in floor_hue_wall_hue_sets_lower[concept[0]]:
                return False
        else:
            if (concept[1] not in floor_hue_wall_hue_sets_upper[concept[0]]):
                return False

        if (concept[1] < 5):
            if concept[2] not in wall_hue_object_hue_sets_lower[concept[1]]:
                return False
        else:
            if (concept[2] not in wall_hue_object_hue_sets_upper[concept[1]]):
                return False

        if (concept[2] < 5):
            if concept[3] not in object_hue_scale_sets_lower[concept[2]]:
                return False
        else:
            if (concept[3] not in object_hue_scale_sets_upper[concept[2]]):
                return False

        if (concept[3] < 4):
            if concept[4] not in scale_shape_sets_lower[concept[3]]:
                return False
        else:
            if (concept[4] not in scale_shape_sets_upper[concept[3]]):
                return False
        return True



    shape_rotation_sets_lower = [
        list(np.random.permutation(9))[:7]
        for _ in range(4)
    ]

    shape_rotation_sets_upper = [
        list(6 + np.random.permutation(9))[:7]
        for _ in range(4)
    ]

    def filter_fn_dep_5(concept):
        ranges = [
            list(range(0, 10)),
            list(range(0, 10)),
            list(range(0, 10)),
            list(range(0, 8)),
            list(range(4)),
            list(range(0, 15)),
        ]

        concept_vector = multiclass_task_bin_concepts_map_fn(concept)

        # First filter as in small dataset to constraint the size of the data a bit
        if not all([
            (concept[i] in ranges[i]) for i in range(len(ranges))
        ]):
            return False

        if (concept[0] < 5):
            if concept[1] not in floor_hue_wall_hue_sets_lower[concept[0]]:
                return False
        else:
            if (concept[1] not in floor_hue_wall_hue_sets_upper[concept[0]]):
                return False

        if (concept[1] < 5):
            if concept[2] not in wall_hue_object_hue_sets_lower[concept[1]]:
                return False
        else:
            if (concept[2] not in wall_hue_object_hue_sets_upper[concept[1]]):
                return False

        if (concept[2] < 5):
            if concept[3] not in object_hue_scale_sets_lower[concept[2]]:
                return False
        else:
            if (concept[3] not in object_hue_scale_sets_upper[concept[2]]):
                return False

        if (concept[3] < 4):
            if concept[4] not in scale_shape_sets_lower[concept[3]]:
                return False
        else:
            if (concept[4] not in scale_shape_sets_upper[concept[3]]):
                return False

        if (concept[4] < 2):
            if concept[5] not in shape_rotation_sets_lower[concept[4]]:
                return False
        else:
            if (concept[5] not in shape_rotation_sets_upper[concept[4]]):
                return False
        return True
else:
    def count_class_balance(y):
        one_hot = tf.keras.utils.to_categorical(y)
        return np.sum(one_hot, axis=0) / one_hot.shape[0]

    def multiclass_binary_concepts_map_fn(concepts):
        new_concepts = np.zeros((concepts.shape[0], 5))
        # We will have 5 concepts:
        # (0) "is it ellipse or square?"
        new_concepts[:, 0] = (concepts[:, 0] < 2).astype(np.int)

        # (1) "is_size < 3?"
        num_sizes = len(set(concepts[:, 1]))
        new_concepts[:, 1] = (concepts[:, 1] < num_sizes/2).astype(np.int)

        # (2) "is rotation < PI/2?"
        num_rots = len(set(concepts[:, 2]))
        new_concepts[:, 2] = (concepts[:, 2] < num_rots/2).astype(np.int)

        # (3) "is x <= 16?"
        num_x_coords = len(set(concepts[:, 3]))
        new_concepts[:, 3] = (concepts[:, 3] < num_x_coords // 2).astype(np.int)

        # (4) "is y <= 16?"
        num_y_coords = len(set(concepts[:, 4]))
        new_concepts[:, 4] = (concepts[:, 4] < num_y_coords // 2).astype(np.int)

        return new_concepts

    def _get_concept_vector(c_data):
        return np.array([
            # First check if it is an ellipse or a square
            int(c_data[0] < 2),
            # Now check that it is "small"
            int(c_data[1] < 3),
            # And it has not been rotated more than PI/2 radians
            int(c_data[2] < 20),
            # Finally, check whether it is in not in the the upper-left quadrant
            int(c_data[3] < 15),
            int(c_data[4] < 15),
        ])

    def multiclass_task_label_fn(c_data):
        # Our task will be a binary task where we are interested in determining
        # whether an image is a "small" ellipse not in the upper-left
        # quadrant that has been rotated less than 3*PI/2 radians
        concept_vector = _get_concept_vector(c_data)
        binary_label_encoding = [
            concept_vector[0] or concept_vector[1],
            concept_vector[2] or concept_vector[3],
            concept_vector[4],
        ]
        return int(
            "".join(list(map(str, binary_label_encoding))),
            2
        )

    def dep_0_filter_fn(concept):
        ranges = [
            list(range(3)),
            list(range(0, 6, 2)),
            list(range(0, 40, 4)),
            list(range(0, 32, 2)),
            list(range(0, 32, 2)),
        ]
        return all([
            (concept[i] in ranges[i]) for i in range(len(ranges))
        ])



    scale_shape_sets_lower = [
        list(np.random.permutation(4))[:3] for i in range(3)
    ]

    scale_shape_sets_upper = [
        list(2 + np.random.permutation(4))[:3] for i in range(3)
    ]
    def dep_1_filter_fn(concept):
        ranges = [
            list(range(3)),
            list(range(6)),
            list(range(0, 40, 4)),
            list(range(0, 32, 2)),
            list(range(0, 32, 2)),
        ]

        concept_vector = _get_concept_vector(concept)

        # First filter as in small dataset to constraint the size of the data a bit
        if not all([
            (concept[i] in ranges[i]) for i in range(len(ranges))
        ]):
            return False
        if concept_vector[0]:
            if concept[1] not in scale_shape_sets_lower[concept[0]]:
                return False
        else:
            if concept[0] not in scale_shape_sets_upper[concept[0]]:
                return False
        return True



    rotation_scale_sets_lower = [
        list(np.random.permutation(30))[:20] for i in range(6)
    ]

    rotation_scale_sets_upper = [
        list(10 + np.random.permutation(30))[:20] for i in range(6)
    ]
    def dep_2_filter_fn(concept):
        ranges = [
            list(range(3)),
            list(range(6)),
            list(range(0, 40, 2)),
            list(range(0, 32, 2)),
            list(range(0, 32, 2)),
        ]

        concept_vector = _get_concept_vector(concept)

        # First filter as in small dataset to constraint the size of the data a bit
        if not all([
            (concept[i] in ranges[i]) for i in range(len(ranges))
        ]):
            return False
        if concept_vector[0]:
            if concept[1] not in scale_shape_sets_lower[concept[0]]:
                return False
        else:
            if concept[0] not in scale_shape_sets_upper[concept[0]]:
                return False

        if concept_vector[1]:
            if concept[2] not in rotation_scale_sets_lower[concept[1]]:
                return False
        else:
            if concept[2] not in rotation_scale_sets_upper[concept[1]]:
                return False
        return True



    x_pos_rotation_sets_lower = [
        list(np.random.permutation(20))[:16]
        for i in range(40)
    ]

    x_pos_rotation_sets_upper = [
        list(12 + np.random.permutation(20))[:16]
        for i in range(40)
    ]
    def dep_3_filter_fn(concept):
        ranges = [
            list(range(3)),
            list(range(6)),
            list(range(0, 40, 2)),
            list(range(0, 32)),
            list(range(0, 32, 2)),
        ]

        concept_vector = _get_concept_vector(concept)

        # First filter as in small dataset to constraint the size of the data a bit
        if not all([
            (concept[i] in ranges[i]) for i in range(len(ranges))
        ]):
            return False
        if concept_vector[0]:
            if concept[1] not in scale_shape_sets_lower[concept[0]]:
                return False
        else:
            if concept[0] not in scale_shape_sets_upper[concept[0]]:
                return False

        if concept_vector[1]:
            if concept[2] not in rotation_scale_sets_lower[concept[1]]:
                return False
        else:
            if concept[2] not in rotation_scale_sets_upper[concept[1]]:
                return False

        if concept_vector[2]:
            if concept[3] not in x_pos_rotation_sets_lower[concept[2]]:
                return False
        else:
            if concept[3] not in x_pos_rotation_sets_upper[concept[2]]:
                return False
        return True

    y_pos_x_pos_sets_lower = [
        list(np.random.permutation(20))[:16]
        for i in range(32)
    ]

    y_pos_x_pos_sets_upper = [
        list(12 + np.random.permutation(20))[:16]
        for i in range(32)
    ]
    def dep_4_filter_fn(concept):
        ranges = [
            list(range(3)),
            list(range(6)),
            list(range(0, 40, 2)),
            list(range(0, 32)),
            list(range(0, 32)),
        ]

        concept_vector = _get_concept_vector(concept)

        # First filter as in small dataset to constraint the size of the data a bit
        if not all([
            (concept[i] in ranges[i]) for i in range(len(ranges))
        ]):
            return False
        if concept_vector[0]:
            if concept[1] not in scale_shape_sets_lower[concept[0]]:
                return False
        else:
            if concept[0] not in scale_shape_sets_upper[concept[0]]:
                return False

        if concept_vector[1]:
            if concept[2] not in rotation_scale_sets_lower[concept[1]]:
                return False
        else:
            if concept[2] not in rotation_scale_sets_upper[concept[1]]:
                return False

        if concept_vector[2]:
            if concept[3] not in x_pos_rotation_sets_lower[concept[2]]:
                return False
        else:
            if concept[3] not in x_pos_rotation_sets_upper[concept[2]]:
                return False

        if concept_vector[3]:
            if concept[4] not in y_pos_x_pos_sets_lower[concept[3]]:
                return False
        else:
            if concept[4] not in y_pos_x_pos_sets_upper[concept[3]]:
                return False
        return True

In [ ]:
_FORCE_RERUN = False
if not USE_DSPRITES:
    _FORCE_RERUN = False
if not USE_DSPRITES:
    def balanced_multiclass_task_bin_concepts_label_fn(concept_dict):
        concept_vector = multiclass_task_bin_concepts_map_fn([
            concept_dict['label_floor_hue'],
            concept_dict['label_wall_hue'],
            concept_dict['label_object_hue'],
            concept_dict['label_scale'],
            concept_dict['label_shape'],
            concept_dict['label_orientation'],
        ])
        if concept_vector[4] == 0:
            offset = 0
            binary_label_encoding = [
                concept_vector[0],
                concept_vector[1],
            ]
        else:
            offset = 4
            binary_label_encoding = [
                concept_vector[2],
                concept_vector[3],
                concept_vector[5],
            ]

        return offset + int(
            "".join(list(map(str, binary_label_encoding))),
            2
        )

    data_x_train, data_y_train, data_c_train = extract_data(
        filter_fn_dep_0,
        dataset_path=os.path.join(DATASETS_DIR, "balanced_multiclass_task_bin_concepts_dep_0_concepts.npz"),
        concept_map_fn=multiclass_task_bin_concepts_map_fn,
        label_fn=balanced_multiclass_task_bin_concepts_label_fn,
        force_rerun=_FORCE_RERUN,
    )

    data_x_train, data_x_test, data_y_train, data_y_test, data_c_train, data_c_test = train_test_split(
        data_x_train,
        data_y_train,
        data_c_train,
        test_size=0.2,
    )

    print("data_x_train.shape =", data_x_train.shape)
    print("data_x_train label distribution:", count_class_balance(data_y_train))
    print("data_x_train concept distribution:", np.sum(data_c_train, axis=0)/data_c_train.shape[0])
    print("data_x_train exclusive concept distribution:", np.sum(data_c_train[np.sum(data_c_train, axis=-1) == 1], axis=0)/data_c_train.shape[0])
    print("data_x_test.shape =", data_x_test.shape)
    print("data_c_test.shape =", data_c_test.shape)
    data_train = (data_x_train, data_y_train, data_c_train)
    data_test = (data_x_test, data_y_test, data_c_test)

    dep_0_corr_mat = np.ones(
        (
            data_train[2].shape[-1],
            len(set(data_train[1]))
        )
    )
    for c in range(dep_0_corr_mat.shape[0]):
        for l in range(dep_0_corr_mat.shape[1]):
            dep_0_corr_mat[c][l] = np.corrcoef(
                data_train[2][:, c],
                (data_train[1] == l).astype(np.int32),
            )[0, 1]
    fig, ax = plt.subplots(1, figsize=(8, 6))
    im, cbar = utils.heatmap(
        np.abs(dep_0_corr_mat),
        [f"$c_{i}$" for i in range(dep_0_corr_mat.shape[0])],
        [f"$l_{i}$" for i in range(dep_0_corr_mat.shape[1])],
        ax=ax,
        cmap="magma",
        cbarlabel=f"Correlation Coef",
        vmin=0, #-1,
        vmax=1,
    )
    texts = utils.annotate_heatmap(im, valfmt="{x:.2f}")
    fig.tight_layout()

    fig.suptitle(f"3dshapes Concept-Label Absolute Correlations ($\lambda = 0$)", fontsize=25)
    fig.subplots_adjust(top=0.85)
    plt.show()

    print("")
else:
    def balanced_multiclass_task_label_fn(c_data):
        # Our task will be a binary task where we are interested in determining
        # whether an image is a "small" ellipse not in the upper-left
        # quadrant that has been rotated less than 3*PI/2 radians
        concept_vector = _get_concept_vector(c_data)
        threshold = 0
        if concept_vector[0] == 1:
            binary_label_encoding = [
                concept_vector[1],
                concept_vector[2],
            ]
        else:
            threshold = 4
            binary_label_encoding = [
                concept_vector[3],
                concept_vector[4],
            ]
        return threshold + int(
            "".join(list(map(str, binary_label_encoding))),
            2
        )

    data_train, data_test = generate_dsprites_dataset(
        label_fn=balanced_multiclass_task_label_fn,
        filter_fn=dep_0_filter_fn,
        dataset_path=os.path.join(DATASETS_DIR, "balanced_multiclass_task_bin_concepts_dep_0_complete_dataset.npz"),
        dsprites_path="dsprites/dsprites_ndarray_co1sh3sc6or40x32y32_64x64.npz",
        concept_map_fn=multiclass_binary_concepts_map_fn,
    )

    dep_0_corr_mat = np.ones(
        (
            data_train[2].shape[-1],
            len(set(data_train[1]))
        )
    )
    for c in range(dep_0_corr_mat.shape[0]):
        for l in range(dep_0_corr_mat.shape[1]):
            dep_0_corr_mat[c][l] = np.corrcoef(
                data_train[2][:, c],
                (data_train[1] == l).astype(np.int32),
            )[0, 1]
    fig, ax = plt.subplots(1, figsize=(8, 6))
    im, cbar = utils.heatmap(
        np.abs(dep_0_corr_mat),
        [f"$c_{i}$" for i in range(dep_0_corr_mat.shape[0])],
        [f"$l_{i}$" for i in range(dep_0_corr_mat.shape[1])],
        ax=ax,
        cmap="magma",
        cbarlabel=f"Correlation Coef",
        vmin=0,
        vmax=1,
    )
    texts = utils.annotate_heatmap(im, valfmt="{x:.2f}")
    fig.tight_layout()

    fig.suptitle(f"dSprites Concept-Label Absolute Correlations ($\lambda = 0$)", fontsize=25)
    fig.subplots_adjust(top=0.85)
    plt.show()

    print("data_dataset train size:", data_train[0].shape[0])
    print("data_dataset train concept size:", data_train[2].shape)
    print("\tTrain balance:", count_class_balance(data_train[1]))
    print("\tConcept balance:", np.sum(data_train[2], axis=0)/data_train[2].shape[0])
    print("")


## Model Construction

In [ ]:
############################################################################
## Build CBM
############################################################################
import concepts_xai.methods.CBM.CBModel as CBM

def construct_cbm(
    encoder,
    decoder,
    num_outputs,
    alpha=0.1,
    learning_rate=1e-3,
    latent_dims=0,
    encoder_output_logits=False,
):
    model_factory = CBM.BypassJointCBM if latent_dims else CBM.JointConceptBottleneckModel
    cbm_model = model_factory(
        encoder=encoder,
        decoder=decoder,
        task_loss=(
            tf.keras.losses.BinaryCrossentropy(from_logits=True) if (num_outputs <= 2)
            else tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
        ),
        name="joint_cbm",
        metrics=[
            tf.keras.metrics.BinaryAccuracy() if (num_outputs <= 2)
            else tf.keras.metrics.SparseTopKCategoricalAccuracy(k=1)
        ],
        alpha=alpha,
        pass_concept_logits=encoder_output_logits,
    )

    ############################################################################
    ## Compile CBM Model
    ############################################################################

    cbm_model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate),
    )
    return cbm_model


## Experiment Loop

In [ ]:
import scipy
import joblib
import concepts_xai.evaluation.metrics.oracle as oracle
import concepts_xai.evaluation.metrics.niching as niching

def construct_trivial_auc_mat(num_concepts):
    result = np.ones((num_concepts, num_concepts), dtype=np.float32) * 0.5
    return result + np.eye(num_concepts, dtype=np.float32) * 0.5
    
    
def cbm_intervention(
    cbm_model,
    x_train,
    x_test,
    y_test,
    c_test,
    intervention_concepts,
):
    # matrix mapping (concept, label) to the value we should set it
    # to during an intervention
    n_concepts = c_test.shape[-1]
    intervention_values = np.zeros((n_concepts, 2))
    if cbm_model.pass_concept_logits:
        # Then the token value we will use for intervening
        # will be based on the percentile of values in the
        # training data
        c_train_pred = cbm_model.encoder(x_train)
        for i in range(n_concepts):
            low_percentile, high_percentile = np.percentile(c_train_pred[:, i], [5, 95])
            intervention_values[i, 0] = low_percentile
            intervention_values[i, 1] = high_percentile
    else:
        # Else we use hard thresholds of 0 and 1 for everything as we are using
        # real probabilities
        intervention_values[:, 1] = 1.0
    
    # Now time to make our concept predictions
    c_test_pred = cbm_model.encoder(x_test).numpy()
    for int_concept in intervention_concepts:
        c_test_pred[:, int_concept] = (
            intervention_values[int_concept, 0] * (c_test[:, int_concept] == 0) +
            intervention_values[int_concept, 1] * (c_test[:, int_concept] == 1)
        )
    
    # Make the actual task predictions
    # Extend c_test_pred so that it has its complement probability as well
    y_test_preds = cbm_model.predict_from_concepts(c_test_pred).numpy()
    y_test_preds = scipy.special.softmax(y_test_preds, axis=-1)
    
    # Time to evaluate these predictions
    return {
        "accuracy": sklearn.metrics.accuracy_score(
            y_test,
            np.argmax(y_test_preds, axis=-1),
        ),
        "auc": sklearn.metrics.roc_auc_score(
            tf.keras.utils.to_categorical(y_test),
            y_test_preds,
            multi_class='ovo',
        ),
    }
    
    
def cbm_mixed_capacity_intervention_experiment_loop(
    train_data,
    test_data,
    experiment_config,
    load_from_cache=False,
):
    experiment_variables = dict(
        task_accuracies=[],
        task_aucs=[],
        concept_accuracies=[],
        oiss=[],
        purity_matrices=[],
        oracle_matrices=[],
        
        intervention_task_accuracies=[],
        intervention_task_aucs=[],

        niss=[],
        
        current_experiment_idx=0,
    )
    experiment_config["data_concepts"] = experiment_config.get(
        "data_concepts",
        experiment_config["num_concepts"],
    )
    
    Path(
        os.path.join(
            experiment_config["results_dir"],
            "models",
        )
    ).mkdir(parents=True, exist_ok=True)
    
    x_train, y_train, c_train = train_data
    x_test, y_test, c_test = test_data
    start_ind = 0
    if load_from_cache:
        cached = os.path.exists(
            os.path.join(experiment_config['results_dir'], 'results.joblib')
        )
        if cached:
            results = joblib.load(
                os.path.join(experiment_config['results_dir'], 'results.joblib')
            )
            start_ind = results['current_experiment_idx']
            complete_cache = True
            for varname in experiment_variables.keys():
                if varname == 'current_experiment_idx':
                    continue
                if (varname not in results):
                    complete_cache = False
                    continue
                if len(results[varname]) != start_ind:
                    complete_cache = False
                    start_ind = min(len(results[varname]), start_ind)
            if complete_cache:
                # Then we are good to go
                if not os.path.exists(
                    os.path.join(experiment_config["results_dir"], "config.yaml")
                ):
                    # then serialize the config as this is a different version run
                    utils.serialize_experiment_config(
                        experiment_config,
                        experiment_config["results_dir"],
                    )
                return results
            else:
                # Else we have a partial initialization so let's use it
                experiment_variables = results
    
    # Let's save our config here either way
    utils.serialize_experiment_config(
        experiment_config,
        experiment_config["results_dir"],
    )
    
    verbosity = experiment_config.get("verbosity", 0)
    for units in experiment_config["model_units"][start_ind:]:
        print("Training with units:", [units, units//2])
        task_accs = []
        concept_accs = []
        aucs = []
        purity_mats = []
        oracle_mats = []
        oiss = []
        non_oracle_purities = []
        niss = []
        
        intervention_task_accuracies = []
        intervention_task_aucs = []
        
        for trial in range(experiment_config["trials"]):
            print(f"\tTrial {trial + 1}/{experiment_config['trials']}")
            # First proceed to do an end-to-end model in case we want to
            # do some task-specific pretraining
            encoder_units = (
                [units, units//2] if experiment_config["encoder_experiment"]
                else experiment_config["encoder_units"]
            )
            decoder_units = (
                [units, units//2] if (not experiment_config["encoder_experiment"])
                else experiment_config["decoder_units"]
            )
            
            end_to_end_model, encoder, decoder = model_utils.construct_end_to_end_model(
                input_shape=experiment_config["input_shape"],
                num_outputs=experiment_config["num_outputs"],
                learning_rate=experiment_config["learning_rate"],
                encoder=model_utils.construct_encoder(
                    input_shape=experiment_config["input_shape"],
                    filter_groups=experiment_config["encoder_filter_groups"],
                    units=encoder_units,
                    concept_cardinality=experiment_config["concept_cardinality"],
                    drop_prob=experiment_config.get("drop_prob", 0.5),
                    max_pool_window=experiment_config.get("max_pool_window", (2, 2)),
                    max_pool_stride=experiment_config.get("max_pool_stride", (2, 2)),
                    latent_dims=experiment_config.get("latent_dims", 0),
                    output_logits=experiment_config.get("encoder_output_logits", False),
                ),
                decoder=model_utils.construct_decoder(
                    units=decoder_units,
                    num_outputs=experiment_config["num_outputs"],
                ),
            )
            
            if experiment_config.get("pre_train_epochs"):
                print("\tModel pre-training...")
                end_to_end_model.fit(
                    x=x_train,
                    y=y_train,
                    epochs=experiment_config["pre_train_epochs"],
                    batch_size=experiment_config["batch_size"],
                    validation_split=experiment_config["holdout_fraction"],
                    verbose=verbosity,
                )
                print("\t\tModel pre-training completed")
            
            # Now time to actually construct and train the CBM
            cbm_model = construct_cbm(
                encoder=encoder,
                decoder=decoder,
                alpha=experiment_config["alpha"],
                learning_rate=experiment_config["learning_rate"],
                num_outputs=experiment_config["num_outputs"],
                latent_dims=experiment_config.get("latent_dims", 0),
                encoder_output_logits=experiment_config.get("encoder_output_logits", False),
            )

            early_stopping_monitor = tf.keras.callbacks.EarlyStopping(
                monitor=experiment_config.get(
                    "early_stop_metric",
                    "val_concept_accuracy",
                ),
                min_delta=experiment_config["min_delta"],
                patience=experiment_config["patience"],
                restore_best_weights=True,
                verbose=2,
                mode=experiment_config.get(
                    "early_stop_mode",
                    "max",
                ),
            )
            if experiment_config["warmup_epochs"]:
                print("\tWarmup training...")
                cbm_model.fit(
                    x=x_train,
                    y=(y_train, c_train),
                    epochs=experiment_config["warmup_epochs"],
                    batch_size=experiment_config["batch_size"],
                    validation_split=experiment_config["holdout_fraction"],
                    verbose=verbosity,
                )
                print("\t\tWarmup training completed")


            print("\tCBM training...")
            cbm_model.fit(
                x=x_train,
                y=(y_train, c_train),
                epochs=experiment_config["max_epochs"],
                batch_size=experiment_config["batch_size"],
                callbacks=[
                    early_stopping_monitor,
                ],
                validation_split=experiment_config["holdout_fraction"],
                verbose=verbosity,
            )
            print("\t\tCBM training completed")
            print("\tSerializing model")
            encoder.save(
                os.path.join(
                    experiment_config["results_dir"],
                    f"models/encoder_{'_'.join(map(str, encoder_units))}_trial_{trial}"
                )
            )
            decoder.save(
                os.path.join(
                    experiment_config["results_dir"],
                    f"models/decoder_{'_'.join(map(str, decoder_units))}_trial_{trial}"
                )
            )
            
            print("\tEvaluating model")
            test_result = cbm_model.evaluate(
                x_test,
                (
                    y_test,
                    c_test[:, :experiment_config["num_concepts"]],
                ),
                verbose=0,
                return_dict=True,
            )
            task_accs.append(
                test_result['sparse_top_k_categorical_accuracy']
                if experiment_config['num_outputs'] > 1 else
                test_result['binary_accuracy']
            )
            concept_accs.append(test_result['concept_accuracy'])
            
            if experiment_config['num_outputs'] > 1:
                # Then lets apply a softmax activation over all the probability
                # classes
                preds = scipy.special.softmax(cbm_model.predict(x_test)[0], axis=-1)

                # And select just the labels that are in fact being used
                one_hot_labels = tf.keras.utils.to_categorical(y_test)
                aucs.append(sklearn.metrics.roc_auc_score(
                    one_hot_labels,
                    preds,
                    multi_class='ovo',
                ))
            else:
                aucs.append(sklearn.metrics.roc_auc_score(
                    y_test,
                    cbm_model.predict(x_test)[0],
                ))
            
            print(
                f"\t\tTest auc = {aucs[-1]:.4f}, "
                f"test concept accuracy = {concept_accs[-1]:.4f}, "
                f"task accuracy = {task_accs[-1]:.4f}"
            )

            print(f"\t\tComputing impurity scores...")
            c_train_pred = cbm_model.encoder(x_train).numpy()
            c_test_pred = cbm_model.encoder(x_test).numpy()

            # Concept niching scores
            nis = niching.niche_impurity_score(
                c_soft=c_test_pred,
                c_true=c_test,
                c_soft_train=c_train_pred,
                c_true_train=c_train,
            )
            
            print("\t\t\tNIS:", nis)
            niss.append(nis)
            
            print("\t\tComputing intervention accuracies...")
            current_intervention_task_acc = []
            current_intervention_task_auc = []
            if experiment_config.get('random_intervention_order', False):
                intervention_order = np.random.permutation(experiment_config['num_concepts'])
            else:
                intervention_order = range(experiment_config['num_concepts'])
            for i in range(len(intervention_order) + 1):
                corrected_concepts = intervention_order[:i]
                intervention_result = cbm_intervention(
                    cbm_model=cbm_model,
                    x_train=x_train,
                    x_test=x_test,
                    y_test=y_test,
                    c_test=c_test[:, :experiment_config["num_concepts"]],
                    intervention_concepts=corrected_concepts,
                )
                current_intervention_task_acc.append(intervention_result['accuracy'])
                current_intervention_task_auc.append(intervention_result['auc'])
            
            print("\t\t\tIntervention AUCs:", current_intervention_task_auc)
            print("\t\t\tIntervention Accuraciess:", current_intervention_task_acc)
            intervention_task_accuracies.append(current_intervention_task_acc)
            intervention_task_aucs.append(current_intervention_task_auc)

            print(f"\t\tComputing OIS...")
            soft_acts = (
                np.concatenate(cbm_model.encoder(x_test), axis=-1)
                if experiment_config["latent_dims"] else c_test_pred
            )
            ois, purity_mat, oracle_mat = oracle.oracle_impurity_score(
                c_soft=soft_acts,
                c_true=c_test,
                output_matrices=True,
            )
            purity_mats.append(purity_mat)
            oracle_mats.append(oracle_mat)
            oiss.append(ois)
            print(f"\t\t\tDone {ois:.4f}")

        task_acc_mean, task_acc_std = np.mean(task_accs), np.std(task_accs)
        experiment_variables["task_accuracies"].append((task_acc_mean, task_acc_std))
        print(f"\tTest task accuracy: {task_acc_mean:.4f} ± {task_acc_std:.4f}")

        concept_acc_mean, concept_acc_std = np.mean(concept_accs), np.std(concept_accs)
        experiment_variables["concept_accuracies"].append((concept_acc_mean, concept_acc_std))
        print(f"\tTest concept accuracy: {concept_acc_mean:.4f} ± {concept_acc_std:.4f}")


        task_auc_mean, task_auc_std = np.mean(aucs), np.std(aucs)
        experiment_variables["task_aucs"].append((task_auc_mean, task_auc_std))
        print(f"\tTest task AUC: {task_auc_mean:.4f} ± {task_auc_std:.4f}")

        purity_mats = np.stack(purity_mats, axis=0)
        purity_mat_mean = np.mean(purity_mats, axis=0)
        purity_mat_std = np.std(purity_mats, axis=0)
        print("\tPurity matrix:")
        for i in range(purity_mat_mean.shape[0]):
            line = "\t\t"
            for j in range(purity_mat_mean.shape[1]):
                line += f'{purity_mat_mean[i, j]:.4f} ± {purity_mat_std[i, j]:.4f}    '
            print(line)

        experiment_variables["purity_matrices"].append((purity_mat_mean, purity_mat_std))


        oracle_mats = np.stack(oracle_mats, axis=0)
        oracle_mat_mean = np.mean(oracle_mats, axis=0)
        oracle_mat_std = np.std(oracle_mats, axis=0)
        print("\tOracle matrix:")
        for i in range(oracle_mat_mean.shape[0]):
            line = "\t\t"
            for j in range(oracle_mat_mean.shape[1]):
                line += f'{oracle_mat_mean[i, j]:.4f} ± {oracle_mat_std[i, j]:.4f}    '
            print(line)

        experiment_variables["oracle_matrices"].append((oracle_mat_mean, oracle_mat_std))

        ois_mean, ois_std = np.mean(oiss), np.std(oiss)
        experiment_variables["oiss"].append((ois_mean, ois_std))
        print(f"\tOIS: {ois_mean:.4f} ± {ois_std:.4f}")

        
        nis_mean, nis_std = np.mean(niss), np.std(niss)
        experiment_variables["niss"].append((nis_mean, nis_std))
        print(f"\tNIS: {nis_mean:.4f} ± {nis_std:.4f}")

        
        # Finally, we end with intervention accuracies
        interventaion_auc_hist = np.stack(intervention_task_aucs, axis=0)
        interventaion_auc_mean = np.mean(interventaion_auc_hist, axis=0)
        interventaion_auc_std = np.std(interventaion_auc_hist, axis=0)
        experiment_variables["intervention_task_aucs"].append((interventaion_auc_mean, interventaion_auc_std))
        line = "\tIntervention AUCs: ["
        for j in range(interventaion_auc_mean.shape[0]):
            line += f'{interventaion_auc_mean[j]:.4f} ± {interventaion_auc_std[j]:.4f},  '
        print(line + "]")
        
        interventaion_acc_hist = np.stack(intervention_task_accuracies, axis=0)
        interventaion_acc_mean = np.mean(interventaion_acc_hist, axis=0)
        interventaion_acc_std = np.std(interventaion_acc_hist, axis=0)
        experiment_variables["intervention_task_accuracies"].append((interventaion_acc_mean, interventaion_acc_std))
        line = "\tIntervention Accuracies: ["
        for j in range(interventaion_acc_mean.shape[0]):
            line += f'{interventaion_acc_mean[j]:.4f} ± {interventaion_acc_std[j]:.4f},  '
        print(line + "]")
        
        # Increase the experiment counter
        experiment_variables["current_experiment_idx"] += 1

        # And serialize the results
        joblib.dump(
            experiment_variables,
            os.path.join(experiment_config["results_dir"], 'results.joblib'),
        )
    return experiment_variables

## Sigmoid Model Experiment

In [ ]:
############################################################################
## Experiment config
############################################################################

base_experiment_config = dict(
    batch_size=64 if USE_DSPRITES else 32,
    max_epochs=100,
    pre_train_epochs=0,
    warmup_epochs=0,
    trials=NUM_TRIALS,
    alpha=1,
    learning_rate=1e-3,
    encoder_filter_groups=[
        [(8, (7, 7))],
        [(16, (5, 5))],
        [(32, (3, 3))],
        [(64, (3, 3))]
    ],
    encoder_units=[64, 64],
   
    model_units=[4, 8, 16, 24, 32, 64],
    encoder_experiment=False,

    latent_decoder_units=[64, 64],
    predictor_max_epochs=100,
    concept_predictor_max_epochs=100,
    
    drop_prob=0.5,
    max_pool_window=(2,2),
    pax_pool_stride=2,
    decoder_units=[64, 64],
    num_outputs=(
        len(set(data_train[1])) if len(set(data_train[1])) > 2
        else 1
    ),
    concept_cardinality=[1 for _ in range(data_train[2].shape[-1])],
    patience=float("inf"),
    min_delta=1e-5,
    results_dir=os.path.join(RESULTS_DIR, "cbm/base_complete"),
    input_shape=data_train[0].shape[1:],
    num_concepts=data_train[2].shape[-1],
    latent_dims=0,
    holdout_fraction=0.1,
    verbosity=0,
    early_stop_metric="val_concept_accuracy",
    early_stop_mode="max",
    encoder_output_logits=False,
    delta_beta=0.05,
)

############################################################################
## Experiment run
############################################################################

base_results = cbm_mixed_capacity_intervention_experiment_loop(
    data_train,
    data_test,
    base_experiment_config,
    load_from_cache=True,
)
print("task_accuracies:", base_results["task_accuracies"])
print("concept_accuracies:", base_results["concept_accuracies"])
print("task_aucs:", base_results["task_aucs"])
print("ois:", base_results["oiss"])
print("nis:", base_results["niss"])


# From Logits CBM Results

In [ ]:
############################################################################
## Experiment config
############################################################################

from_logits_experiment_config = dict(
    batch_size=64 if USE_DSPRITES else 32,
    max_epochs=100,
    pre_train_epochs=0,
    warmup_epochs=0,
    trials=NUM_TRIALS,
    alpha=1,
    learning_rate=1e-3,
    encoder_filter_groups=[
        [(8, (7, 7))],
        [(16, (5, 5))],
        [(32, (3, 3))],
        [(64, (3, 3))]
    ],
    encoder_units=[64, 64],
   
    model_units=[4, 8, 16, 24, 32, 64],
    encoder_experiment=False,

    latent_decoder_units=[64, 64],
    predictor_max_epochs=100,
    concept_predictor_max_epochs=100,
    
    drop_prob=0.5,
    max_pool_window=(2,2),
    pax_pool_stride=2,
    decoder_units=[64, 64],
    num_outputs=(
        len(set(data_train[1])) if len(set(data_train[1])) > 2
        else 1
    ),
    concept_cardinality=[1 for _ in range(data_train[2].shape[-1])],
    patience=float("inf"),
    min_delta=1e-5,
    results_dir=os.path.join(RESULTS_DIR, "cbm/from_logits_complete"),
    input_shape=data_train[0].shape[1:],
    num_concepts=data_train[2].shape[-1],
    latent_dims=0,
    holdout_fraction=0.1,
    verbosity=0,
    early_stop_metric="val_concept_accuracy",
    early_stop_mode="max",
    encoder_output_logits=True,
    delta_beta=0.05,
)

############################################################################
## Experiment run
############################################################################

from_logits_results = cbm_mixed_capacity_intervention_experiment_loop(
    data_train,
    data_test,
    from_logits_experiment_config,
    load_from_cache=True,
)
print("task_accuracies:", from_logits_results["task_accuracies"])
print("concept_accuracies:", from_logits_results["concept_accuracies"])
print("task_aucs:", from_logits_results["task_aucs"])
print("oiss:", from_logits_results["oiss"])
print("niss:", from_logits_results["niss"])


# Visualise Results

In [ ]:
LATEX_SYMBOL = ""

def bold_text(x):
    if LATEX_SYMBOL == "$":
        return r"$\textbf{" + x + "}$"
    return x

def plot_intervention_results(
    results_1,
    cand_1_idx,
    cand_1_name,
    results_2,
    cand_2_idx,
    cand_2_name,
    num_concepts=5,
    savepath=None,
    show_impurity=True,
    start_y=50,
    scatter_start_y=65,
    delta_y=5,
    max_y=100,
    scatter_max_y=100,
    fig_width=38,
    fig_height=8,
    legend_fontsize=30,
    impurity_start_y=0,
    impurity_delta_y=5,
    impurity_max_y=85,
    axis_fontsize=23,
):
    clrs = sns.color_palette("tab10", 10)
    color_map = {}
    scale = 1
    fig, axs = plt.subplots(1, 3 if show_impurity else 2, figsize=(fig_width, fig_height))
    ax = axs[0]
    x_labels = ['Task', 'Concept']
    y_vals_cand_1 = [
        # The task accuracy
        np.array(results_1['task_accuracies'][cand_1_idx][0]) * 100,
        # The concept accuracy
        np.array(results_1['concept_accuracies'][cand_1_idx][0]) * 100,
    ]
    y_errs_cand_1 = [
        # The task accuracy
        np.array(results_1['task_accuracies'][cand_1_idx][1]) * 100,
        # The concept accuracy
        np.array(results_1['concept_accuracies'][cand_1_idx][1]) * 100,
    ]

    y_vals_cand_2 = [
        # The task accuracy
        np.array(results_2['task_accuracies'][cand_2_idx][0]) * 100,
        # The concept accuracy
        np.array(results_2['concept_accuracies'][cand_2_idx][0]) * 100,
    ]
    y_errs_cand_2 = [
        # The task accuracy
        np.array(results_2['task_accuracies'][cand_2_idx][1]) * 100,
        # The concept accuracy
        np.array(results_2['concept_accuracies'][cand_2_idx][1]) * 100,
    ]

    all_models = [
        (cand_1_name, y_vals_cand_1, y_errs_cand_1),
        (cand_2_name, y_vals_cand_2, y_errs_cand_2),
    ]
    num_models = len(all_models) + 1
    center = np.zeros(len(x_labels))
    for i, (method_name, accs, stds) in enumerate(all_models):
        accs = np.array(accs)
        stds = np.array(stds)
        print(method_name, accs, stds)
        print("accs =", accs)
        print("stds =", stds)
        if method_name not in color_map:
            color_map[method_name] = clrs[len(color_map)]
        color = color_map[method_name]
        x_coords = scale * (np.arange(0, len(x_labels)) - (1/2 - 1/(2 * num_models)) + i/num_models)
        center += x_coords/len(all_models)
        ax.bar(
            x_coords,
            accs,
            width=scale/num_models,
            color=color,
            align='center',
            label=method_name,
            yerr=2*stds,
            capsize=5,
            edgecolor="black",
        )
    ax.set_ylabel("Accuracy (\%)", fontsize=axis_fontsize)
    ax.set_xlabel("")
    ax.set_title(bold_text("Accuracy Comparison"), fontsize=axis_fontsize)
    ax.set_xticks(center)
    ax.set_xticklabels(x_labels, fontsize=20)
    ax.set_yticks(np.arange(start_y, max_y+delta_y, delta_y))
    ax.set_yticklabels(list(map(lambda x: f"{x}", np.arange(start_y, max_y + delta_y, delta_y))), fontsize=18)
    ax.set_ylim((start_y, max_y + delta_y))
    ax.grid(False)
    handles, labels = ax.get_legend_handles_labels()

    if show_impurity:
        start_y = impurity_start_y
        delta_y = impurity_delta_y
        max_y = impurity_max_y
        ax = axs[1]
        x_labels = [
            'OIS',
            'NIS',
        ]
        y_vals_cand_1 = [
            # The OIS
            np.array(results_1['oiss'][cand_1_idx][0]) * 100,
            # The NIS
            np.array(results_1['niss'][cand_1_idx][0]) * 100,
        ]
        y_errs_cand_1 = [
            # The OIS
            np.array(results_1['oiss'][cand_1_idx][1]) * 100,
            # The NIS
            np.array(results_1['niss'][cand_1_idx][1]) * 100,
        ]

        y_vals_cand_2 = [
            # The OIS
            np.array(results_2['oiss'][cand_2_idx][0]) * 100,
            # The NIS
            np.array(results_2['niss'][cand_2_idx][0]) * 100,
        ]
        y_errs_cand_2 = [
            # The OIS
            np.array(results_2['oiss'][cand_2_idx][1]) * 100,
            # The NIS
            np.array(results_2['niss'][cand_2_idx][1]) * 100,
        ]

        all_models = [
            (cand_1_name, y_vals_cand_1, y_errs_cand_1),
            (cand_2_name, y_vals_cand_2, y_errs_cand_2),
        ]
        num_models = len(all_models) + 1
        center = np.zeros(len(x_labels))
        for i, (method_name, accs, stds) in enumerate(all_models):
            accs = np.array(accs)
            stds = np.array(stds)
            print(method_name, accs, stds)
            if method_name not in color_map:
                color_map[method_name] = clrs[len(color_map)]
            color = color_map[method_name]
            x_coords = scale * (np.arange(0, len(x_labels)) - (1/2 - 1/(2 * num_models)) + i/num_models)
            center += x_coords/len(all_models)
            ax.bar(
                x_coords,
                accs,
                width=scale/num_models,
                color=color,
                align='center',
                label=method_name,
                yerr=2*stds,
                capsize=5,
                edgecolor="black",
            )
        ax.set_ylabel("Score (\%)", fontsize=axis_fontsize)
        ax.set_xlabel("")
        ax.set_title(bold_text("Score Comparison"), fontsize=axis_fontsize)
        ax.set_xticks(center)
        ax.set_xticklabels(x_labels, fontsize=20)
        ax.set_yticks(np.arange(start_y, max_y+delta_y, delta_y))
        ax.set_yticklabels(list(map(lambda x: f"{x}", np.arange(start_y, max_y + delta_y, delta_y))), fontsize=18)
        ax.set_ylim((start_y, max_y + delta_y))
        ax.grid(False)
        handles, labels = ax.get_legend_handles_labels()

    ax = axs[2 if show_impurity else 1]
    x_labels = list(map(str, np.arange(0, num_concepts + 1)))
    x_coords = np.arange(0, num_concepts + 1)
    y_vals_cand_1 = 100 * np.array(results_1['intervention_task_accuracies'][cand_1_idx][0])
    y_errs_cand_1 = 100 * np.array(results_1['intervention_task_accuracies'][cand_1_idx][1])

    y_vals_cand_2 = 100 * np.array(results_2['intervention_task_accuracies'][cand_2_idx][0])
    y_errs_cand_2 = 100 * np.array(results_2['intervention_task_accuracies'][cand_2_idx][1])

    all_models = [
        (cand_1_name, y_vals_cand_1, y_errs_cand_1),
        (cand_2_name, y_vals_cand_2, y_errs_cand_2),
    ]
    num_models = len(all_models) + 1
    for i, (method_name, accs, stds) in enumerate(all_models):
        accs = np.array(accs)
        stds = np.array(stds)
        print(method_name, accs, stds)
        if method_name not in color_map:
            color_map[method_name] = clrs[len(color_map)]
        color = color_map[method_name]
        ax.plot(
            x_coords,
            accs,
            c=color,
            zorder=1,
        )
        plt.scatter(
            x_coords,
            accs,
            s=50,
            color=color,
            label=method_name,
            zorder=2,
        )
        ax.fill_between(
            x_coords,
            accs - stds,
            accs + stds,
            alpha=0.3,
            facecolor=color,
        )
    ax.set_ylabel("Test Accuracy (\%)", fontsize=axis_fontsize)
    ax.set_xlabel("Concepts Intervened", fontsize=axis_fontsize)
    ax.set_title(bold_text("Effects of Intervention"), fontsize=axis_fontsize)
    ax.set_xticks(x_coords)
    ax.set_xticklabels(x_labels, fontsize=20)
    ax.set_yticks(np.arange(scatter_start_y, scatter_max_y+delta_y, delta_y))
    ax.set_yticklabels(list(map(lambda x: f"{x}", np.arange(scatter_start_y, scatter_max_y + delta_y, delta_y))), fontsize=18)
    ax.set_ylim((scatter_start_y, scatter_max_y + delta_y))
    ax.grid(False)

    lgd = fig.legend(handles, labels, fontsize=legend_fontsize, loc='upper center', bbox_to_anchor=(0.5,0.03), ncol=(num_models - 1))
    if savepath:
        plt.savefig(
            savepath,
            bbox_extra_artists=(lgd,),
            bbox_inches='tight'
        )
    plt.show()

In [ ]:
cbm_intervention_capacity_shapes3d = {}
cbm_intervention_capacity_shapes3d["base"] = joblib.load(os.path.join(
    "intervention_results/dsprites",
    "cbm/base_complete/results.joblib"
))
cbm_intervention_capacity_shapes3d["logits"] = joblib.load(os.path.join(
    "intervention_results/dsprites",
    "cbm/from_logits_complete/results.joblib"
))

In [ ]:
capacities = [4, 8, 16, 24, 32]
cand_1_idx = 2
cand_1_set = "logits"
cand_2_idx = 2
cand_2_set = "base"
plot_intervention_results(
    results_1=cbm_intervention_capacity_shapes3d[cand_1_set],
    cand_1_idx=cand_1_idx,
    cand_1_name=f"Joint-CBM{'-Logits' if cand_1_set == 'logits' else ''}",
    results_2=cbm_intervention_capacity_shapes3d[cand_2_set],
    cand_2_idx=cand_2_idx,
    cand_2_name=f"Joint-CBM{'-Logits' if cand_2_set == 'logits' else ''}",
    num_concepts=5,
    delta_y=10,
    fig_width=17,
    fig_height=4,
    legend_fontsize=20,
    impurity_delta_y=10,
    axis_fontsize=19,
)

In [ ]:
capacities = [4, 8, 16, 24, 32]
cand_2_idx = 4
cand_2_set = "logits"
cand_1_idx = 1
cand_1_set = cand_1_set
plot_intervention_results(
    results_1=cbm_intervention_capacity_shapes3d[cand_1_set],
    cand_1_idx=cand_1_idx,
    cand_1_name=f"Joint-CBM{'-Logits' if cand_1_set == 'logits' else ''} (Label Predictor MLP Layers [{capacities[cand_1_idx]}, {capacities[cand_1_idx]//2}])",
    results_2=cbm_intervention_capacity_shapes3d[cand_2_set],
    cand_2_idx=cand_2_idx,
    cand_2_name=f"Joint-CBM{'-Logits' if cand_2_set == 'logits' else ''} (Label Predictor MLP Layers [{capacities[cand_2_idx]}, {capacities[cand_2_idx]//2}])",
    num_concepts=5,
    delta_y=10,
    fig_width=18,
    fig_height=4,
    legend_fontsize=20,
    impurity_delta_y=10,
    axis_fontsize=18,
)